In [1]:
import os
import re
import sys
import math 
import fiona
import random
import logging
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
import skimage
import skimage.draw
import pyproj
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_KEA.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_KEA.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_KEA.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_KEA.dll
127: The specified procedure could not be found.



In [45]:
def get_grid_idx(cell_size=64, height=2357, width=1892):
    num_h = math.ceil(height / cell_size)
    num_w = math.ceil(width / cell_size)
    num_range = np.arange(num_h * num_w).reshape(num_h, num_w)
    print(f'There are {num_h * num_w} grid groups in total.')
    grid_idx = num_range.repeat(cell_size, axis=0).repeat(cell_size, axis=1)
    print(f'before: {grid_idx.shape}')
    grid_idx = grid_idx[:height, :width]
    print(f'after: {grid_idx.shape}')
    return grid_idx

In [46]:
grid_idx = get_grid_idx()

There are 1110 grid groups in total.
before: (2368, 1920)
after: (2357, 1892)


In [47]:
1110*0.2

222.0

In [51]:
grid_idx_test = random.sample(list(np.arange(1110)), math.ceil(1110*0.2))
grid_idx_train_val = list(set(np.arange(1110)) - set(grid_idx_test))

In [53]:
pred_path = '../preds/1008-183014_rfc.tif'
band_pred, meta_pred = load_geotiff(pred_path)

In [66]:
def visualize_train_test_grid_split(meta_src, grid_idx_test, grid_idx_train_val, spatial_dict, save_path):
    # get the grid idx
    cell_size, height, width = spatial_dict['cell_size'], spatial_dict['height'], spatial_dict['width']
    grid_idx = get_grid_idx(cell_size, height, width).reshape(-1)
    # build 
    output = np.zeros_like(grid_idx)
    test_mask = [True if idx in grid_idx_test else False for idx in grid_idx]
    train_val_mask = [True if idx in grid_idx_train_val else False for idx in grid_idx]
    output[test_mask] = 1
    output[train_val_mask] = 2
    
    with rasterio.Env():
        # Write an array as a raster band to a new 8-bit file. We start with the profile of the source
        out_meta = meta_src.copy()
        out_meta.update(
            dtype=rasterio.uint8,
            count=1,
            compress='lzw')
        with rasterio.open(save_path, 'w', **out_meta) as dst:
            # reshape into (band, height, width)
            dst.write(output.reshape(1, out_meta['height'], out_meta['width']).astype(rasterio.uint8))
            dst.write_colormap(
            1, {
                0: (0, 0, 0), 
                1: (240, 72, 72),
                2: (71, 208, 240)})
            
def load_geotiff(path, window=None):
    """ Load the geotiff as a list of numpy array.
        INPUT : path (str) -> the path to the geotiff
                window (rasterio.windows.Window) -> the window to use when loading the image
        OUTPUT : band (list of numpy array) -> the different bands as float scaled to 0:1
                 meta (dictionary) -> the metadata associated with the geotiff
    """
    with rasterio.open(path) as f:
        band = [skimage.img_as_float(f.read(i+1, window=window)) for i in range(f.count)]
        meta = f.meta
        if window is not None:
            meta['height'] = window.height
            meta['width'] = window.width
            meta['transform'] = f.window_transform(window)
    return band, meta

In [67]:
spatial_dict = {'cell_size': 64, 'height': meta_pred['height'], 'width': meta_pred['width']}
visualize_train_test_grid_split(meta_pred, grid_idx_test, grid_idx_train_val, spatial_dict, './train_test_split.tiff')

There are 1110 grid groups in total.
before: (2368, 1920)
after: (2357, 1892)
